# The Search Algorithm

Here I will create the algorithm that decides which variations to play. I will be using a min max algorithm. This is basically an algorithm that skips variations that can't affect the algorithm.

Lets assume we are playing as white. The idea is that in each step (turn for a player) the algorithm will either minimize the eval or maximize it. Variations can be seen as trees (with corresponding roots, nodes and leafs). Suppose we evaluate to depth = n, where it is blacks turn. The algorithm will start by evaluating all the leafs of one of the previous nodes, since it is black's turn we assume black will try to play the best move (minimize evaluation). Therefore from that set of leafs, the only relevant leaf is the one with minimum evaluation. However we will chose the node that has min(set of leafs for node) the maximum from all nodes. This means that when evaluating the leafs of the next node we can stop (and skip to the next set of leafs for another node) whenever we find a leaf whose evaluation is less than the minimum at that point between all the leafs evaluated.

We can do this for each level, starting from the leafs of the tree upward toward the root.

I will start giving a fen to the algorithm as the input. For finding the variation I will use the Python - Chess Library. 

THINGS To do:
* Make engine think while it ins't its move (using the i's in alpha_beta algorithm) - **Done on algorithm, need to finish on engine.**
* Move ordering; consider best moves first. If best moves are considered first the alpha beta algorithm will be much faster (it breaks more times). I start by doing null move pruning.
* Use Pypy interpreter instead of CPython, since it is several magnitudes faster (except for numpy, so maybe we should avoid numpy).


**I have been using the python-chess library to compute the legal moves every time a move is made, however it is faster to precompute all the possible moves beforehand and store them in a table**

In [2]:
import numpy as np
import chess
from functools import partial # To preassign arguments of fucntions, creating a new function

In [3]:
pip install matplotlib

Note: you may need to restart the kernel to use updated packages.


In [4]:
import time

# Test mod 8 methods
start_time = time.time()
for i in range(64):
    i % 8
end_time = time.time()
print(start_time-end_time)

start_time = time.time()
for i in range(64):
    divmod(i,8)
end_time = time.time()
print(start_time-end_time)

# Test mod 10 methods
start_time = time.time()
for i in range(64):
    i%10
end_time = time.time()
print(start_time-end_time)

start_time = time.time()
for i in range(64):
    divmod(i,10)
end_time = time.time()
print(start_time-end_time)

start_time = time.time()
for i in range(64):
    int(str(i)[-1])
end_time = time.time()
print(start_time-end_time)


-4.100799560546875e-05
-4.7206878662109375e-05
-2.8848648071289062e-05
-2.9802322387695312e-05
-4.887580871582031e-05


The fastest is i%10 however i%8 is also pretty fast, also it depends on the interpreter so it would be reasonable to try this with Pypy too. I will choose i%8, therefore my position will be a list of 64 elements. **Taking 100 has one further advantage, we are creating a 10 by 10 board and when checking if pieces are at the end of the board this is useful**. I will be dividing the board in 4 squares and precomputing the moves on one of the 4 by 4 squares say (a1, a4, d1,d4). And using the symmetries I will be able to precompute all possible moves.


# Lists of possible moves for each square


In [295]:
N = 8
E = 1
S = -8
W = -1
directions_a1 = {#a1
    "P": {N, N+E, N+N, N+W}, # pawn will be dealt with on class since it depends on other things
    "N": {N+N+E, N+E+E}, # knight
    "B": {N+E:7}, #length of movement will be dealt inside class since it depends on other things
    "R": {N:7, E:7},
    "Q": {N:7, E:7, N+E:7},
    "K": {N, E, N+E},}
directions_b1 = { #b1
    "P": {N, N+E, N+N, N+W}, 
    "N": {N+N+E, N+E+E, N+N+W}, 
    "B": {N+E:6, N+W:1},
    "R": {N:7, E:6, W:1},
    "Q": {N:7, E:6, N+E:6, N+W:1, W:1},
    "K": {N, E, N+E, N+W, W},}
directions_a2 = { #a2
    "P": {N, N+E, N+N, N+W}, 
    "N": {N+N+E, N+E+E, S+E+E}, 
    "B": {N+E:6, S+E:1},
    "R": {N:6, E:7, S:1},
    "Q": {N:6, E:7, N+E:6, S:1, S+E:1},
    "K": {N, E, N+E, S, S+E},}
directions_c1 = { #c1
    "P": {N, N+E, N+N, N+W}, 
    "N": {N+N+E, N+E+E, N+N+W, N+W+W}, 
    "B": {N+E:5, N+W:2},
    "R": {N:7, E:5, W:2},
    "Q": {N:7, E:5, N+E:5, W:2, N+W:2},
    "K": {N, E, N+E, W, N+W},}
directions_d1 = { #d1
    "P": {N, N+E, N+N, N+W}, 
    "N": {N+N+E, N+E+E, N+N+W, N+W+W}, 
    "B": {N+E:4, N+W:3},
    "R": {N:7, E:4, W:3},
    "Q": {N:7, E:4, N+E:4, W:3, N+W:3},
    "K": {N, E, N+E, W, N+W},}
directions_a3 = { #a3
    "P": {N, N+E, N+N, N+W},
    "N": {N+N+E, N+E+E, S+E+E, S+S+E},
    "B": {N+E:5, S+E:2},
    "R": {N:5, E:7, S:2},
    "Q": {N:5, E:7, N+E:5, S:2, S+E:2},
    "K": {N, E, N+E, S, S+E},}
directions_a4 = { #a4 
    "P": {N, N+E, N+N, N+W},
    "N": {N+N+E, N+E+E, S+E+E, S+S+E},
    "B": {N+E:4, S+E:3},
    "R": {N:4, E:7, S:3},
    "Q": {N:4, E:7, N+E:4, S:3, S+E:3},
    "K": {N, E, N+E, S, S+E},}
directions_b2 = { #b2
    "P": {N, N+E, N+N, N+W},
    "N": {N+N+E, N+N+W, N+E+E, S+E+E},
    "B": {N+E:6, N+W:1, S+E:1, S+W:1},
    "R": {N:6, E:6, S:1, W:1},
    "Q": {N:6, E:6, S:1, W:1, N+E:6, S+E:1, S+W:1, N+W:1},
    "K": {N, E, S, W, N+E, S+E, S+W, N+W},}
directions_b3 = { #b3
    "P": {N, N+E, N+N, N+W},
    "N": {N+N+E, N+N+W, N+E+E, S+E+E, S+S+E, S+S+W},
    "B": {N+E:5, N+W:1, S+E:2, S+W:1},
    "R": {N:5, E:6, S:2, W:1},
    "Q": {N:5, E:6, S:2, W:1, N+E:5, S+E:2, S+W:1, N+W:1},
    "K": {N, E, S, W, N+E, S+E, S+W, N+W},}
directions_c2 = { #c2
    "P": {N, N+E, N+N, N+W},
    "N": {N+N+E,N+N+W, N+E+E, S+E+E, N+W+W, S+W+W},
    "B": {N+E:5, N+W:2, S+E:1, S+W:1},
    "R": {N:6, E:5, S:1, W:2},
    "Q": {N:6, E:5, S:1, W:2, N+E:5, S+E:1, S+W:1, N+W:2},
    "K": {N, E, S, W, N+E, S+E, S+W, N+W},}
directions_c3 = { #centre squares c3
    "P": {N, N+E, N+N, N+W},
    "N": {N+N+E, N+E+E, S+E+E, S+S+E, S+S+W, S+W+W, N+W+W, N+N+W},
    "B": {N+E:5, N+W:2, S+E:2, S+W:2},
    "R": {N:5, E:5, S:2, W:2},
    "Q": {N:5, E:5, S:2, W:2, N+E:5, S+E:2, S+W:2, N+W:2},
    "K": {N, E, S, W, N+E, S+E, S+W, N+W},}
directions_d3 = { #centre squares d3
    "P": {N, N+E, N+N, N+W},
    "N": {N+N+E, N+E+E, S+E+E, S+S+E, S+S+W, S+W+W, N+W+W, N+N+W},
    "B": {N+E:4, N+W:3, S+E:2, S+W:2},
    "R": {N:5, E:4, S:2, W:3},
    "Q": {N:5, E:4, S:2, W:3, N+E:4, S+E:2, S+W:2, N+W:3},
    "K": {N, E, S, W, N+E, S+E, S+W, N+W},}
directions_c4 = { #centre squares c4
    "P": {N, N+E, N+N, N+W},
    "N": {N+N+E, N+E+E, S+E+E, S+S+E, S+S+W, S+W+W, N+W+W, N+N+W},
    "B": {N+E:4, N+W:2, S+E:3, S+W:2},
    "R": {N:4, E:5, S:3, W:2},
    "Q": {N:4, E:5, S:3, W:2, N+E:4, S+E:3, S+W:2, N+W:2},
    "K": {N, E, S, W, N+E, S+E, S+W, N+W},}
directions_d4 = { #centre squares d4
    "P": {N, N+E, N+N, N+W}, 
    "N": {N+N+E, N+E+E, S+E+E, S+S+E, S+S+W, S+W+W, N+W+W, N+N+W}, 
    "B": {N+E:4, N+W:3, S+E:3, S+W:3},
    "R": {N:4, E:4, S:3, W:3},
    "Q": {N:4, E:4, S:3, W:3, N+E:4, S+E:3, S+W:3, N+W:3},
    "K": {N, E, S, W, N+E, S+E, S+W, N+W},}
directions_b4 = { #b4
    "P": {N, N+E, N+N, N+W}, 
    "N": {N+N+E, N+E+E, S+E+E, S+S+E, S+S+W, N+N+W}, 
    "B": {N+E:4, N+W:1, S+E:3, S+W:1},
    "R": {N:4, E:7, S:3, W:1},
    "Q": {N:4, E:7, S:3, W:1, N+E:4, S+E:3, S+W:1, N+W:1},
    "K": {N, E, S, W, N+E, S+E, S+W, N+W},}
directions_d2 = { #d2
    "P": {N, N+E, N+N, N+W}, 
    "N": {N+N+E, N+E+E, S+E+E, S+W+W, N+W+W, N+N+W}, 
    "B": {N+E:4, N+W:3, S+E:1, S+W:1},
    "R": {N:6, E:4, S:1, W:3},
    "Q": {N:6, E:4, S:1, W:3, N+E:4, S+E:1, S+W:1, N+W:3},
    "K": {N, E, S, W, N+E, S+E, S+W, N+W},}


In [296]:
DirectionsDict = {0: directions_a1, 1: directions_b1,
                  8: directions_a2, 2: directions_c1, 3:directions_d1,
                  16: directions_a3, 24: directions_a4, 9: directions_b2,
                  17: directions_b3, 25: directions_b4, 10: directions_c2, 
                  11: directions_d2, 18: directions_c3, 19: directions_d3,
                  26: directions_c4, 27: directions_d4}
DirectionsDict[1]


{'P': {7, 8, 9, 16},
 'N': {10, 15, 17},
 'B': {9: 6, 7: 1},
 'R': {8: 7, 1: 6, -1: 1},
 'Q': {8: 7, 1: 6, 9: 6, 7: 1, -1: 1},
 'K': {-1, 1, 7, 8, 9}}

In [297]:
board = [ 'R', 'N', 'B', 'Q', 'K', 'B', 'N', 'R',          
         'P', 'P', 'r', 'P', 'klk', 'P', 'P', 'P',          
         '0', '0', '0', '0', '0', '0', '0', '0',          
         '0', '0', 'k', '0', '0', '0', '0', '0',       
         '0', '0', '0', '0', '0', '0', '0', '0',          
         '0', '2', 'm', '0', '0', '0', '0', '0',          
         'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p',          
         'r', 'n', 'b', 'q', 'k', 'b', 'n', 'k' ]

# Quadrant h8
flipboard = list(reversed(board)) # Flip the board pieces horizontally
flipsquarelist = list(reversed(range(64)))# correct
flipsquare = flipsquarelist[3]
print('Quadrant_h8:', flipboard) # Este esta hecho para h8


# Quadrant a8
flipboard = [square for row in [board[i:i+8] for i in range(0, 64, 8)][::-1] for square in row]
flipsquarelist = [square for row in [list(range(64))[::-1][i:i+8][::-1] for i in range(0, 64, 8)] for square in row]
flipsquare = flipsquarelist[3]
print('Quadrant_a8:', flipboard)

# Quadrant h1
flipboard = [square for row in [board[i:i+8][::-1] for i in range(0, 64, 8)] for square in row] 
flipsquarelist = [square for row in [list(range(64))[i:i+8][::-1] for i in range(0, 64, 8)] for square in row]
flipsquare = flipsquarelist[3]
print('Quadrant_h1:', flipboard) # Este esta hecho para h1


Quadrant_h8: ['k', 'n', 'b', 'k', 'q', 'b', 'n', 'r', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', '0', '0', '0', '0', '0', 'm', '2', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', 'k', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', 'P', 'P', 'P', 'klk', 'P', 'r', 'P', 'P', 'R', 'N', 'B', 'K', 'Q', 'B', 'N', 'R']
Quadrant_a8: ['r', 'n', 'b', 'q', 'k', 'b', 'n', 'k', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', '0', '2', 'm', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', 'k', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', 'P', 'P', 'r', 'P', 'klk', 'P', 'P', 'P', 'R', 'N', 'B', 'Q', 'K', 'B', 'N', 'R']
Quadrant_h1: ['R', 'N', 'B', 'K', 'Q', 'B', 'N', 'R', 'P', 'P', 'P', 'klk', 'P', 'r', 'P', 'P', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', 'k', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', 'm', '2', '0', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'k', 'n', 'b', 'k', 'q', 'b', 'n

In [298]:
def pretty_print(board):
    # Create a list of rank labels (1 to 8) and file labels (a to h)
    ranks = [str(i) for i in range(8, 0, -1)]
    files = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h']


    # Print each rank of the board
    for i in range(8):
        rank = ''.join(str(board[(8-i-1) * 8 + j]) for j in range(8))
        rank = rank.replace('0', '.')
        rank = rank.replace('K', '♔')
        rank = rank.replace('Q', '♕')
        rank = rank.replace('R', '♖')
        rank = rank.replace('B', '♗')
        rank = rank.replace('N', '♘')
        rank = rank.replace('P', '♙')
        rank = rank.replace('k', '♚')
        rank = rank.replace('q', '♛')
        rank = rank.replace('r', '♜')
        rank = rank.replace('b', '♝')
        rank = rank.replace('n', '♞')
        rank = rank.replace('p', '♟')
        print(rank )

    

In [736]:
#Position Class
from collections import namedtuple
from collections import Counter

Move = namedtuple("Move", "i j prom score") # The Move class is a named tuple that stores information about a
                                      # move in the chess game. It has three fields: i and j, which are 
                                      # the starting and ending positions of the piece being moved, respectively, 
                                      # and prom, which is the promotion piece if the move is a pawn promotion.
                                      # and score +2 captures
                                                # +1 checks
import copy

class Position:
    def __init__(self, board, psquare, wc, bc, history, turn): # When creating a Position we will have to specify these variables
        self.board = board # an array of length 64
        self.psquare = psquare # An integer representing the square that can be captured using en passant.
                               # It will be -1 if the last move wasn't a double pawn move.
                               # I will change the passant variable every time a double pawn move is made in 
                               # the move method.
        self.wc = wc
        self.bc = bc # The castling rights, these will be lists of two Boolean variables. The first element 
                     # of the list represents kingside castle, the second queenside castle. They only check 
                     # if the king or rook has already moved (to check if there are in between checks we do
                     # it in gen_moves method).
        self.history = [''.join(self.board)]
        self.turn = turn # True if white, False if black 
    def checks_pins(self):
        checks = [0]
        pins = {}
        for square, piece in enumerate(self.board):
            if piece not in 'prqnb': # We check for all opponents pieces that can give checks (not kings)
                continue 
            downwards = False
            flipsquare = square # flipsquare is the corresponding square in the first quarterboard
            flipsquarelist = list(range(64))
            flipboard = self.board
            if any(square in range(start, start+4) for start in [4, 12, 20, 28]): 
                # h1 quadrant
                flipboard = [square for row in [self.board[i:i+8][::-1] for i in range(0, 64, 8)] for square in row] 
                flipsquarelist = [square for row in [list(range(64))[i:i+8][::-1] for i in range(0, 64, 8)] for square in row]
                flipsquare = flipsquarelist[square]                 
            elif any(square in range(start, start+4) for start in [36, 44, 52, 60]): # h8 quadrant
                # Quadrant h8
                downwards = True
                flipboard = list(reversed(self.board)) # Flip the board pieces horizontally
                flipsquarelist = list(reversed(range(64)))# correct
                flipsquare = flipsquarelist[square]
            elif any(square in range(start, start+4) for start in [32, 40, 48, 56]): # a8 quadrant
                # Quadrant a8
                downwards = True
                flipboard = [square for row in [self.board[i:i+8] for i in range(0, 64, 8)][::-1] for square in row]
                flipsquarelist = [square for row in [list(range(64))[::-1][i:i+8][::-1] for i in range(0, 64, 8)] for square in row]
                flipsquare = flipsquarelist[square]
            if piece in 'rqb': # Opponents sliders
                for d in DirectionsDict[flipsquare][piece.upper()].keys():
                    squareslist = [square]
                    count = 0
                    outer_break = False
                    for k in range(DirectionsDict[flipsquare][piece.upper()][d]):
                        jflip = flipsquare+(k+1)*d
                        j = flipsquarelist[jflip]
                        q = flipboard[jflip] # What is on destination square
                        if q.islower(): # Own piece is blocking
                            break
                        if q.isupper() and q!= 'K':
                            count += 1
                            pinned_square = j
                        if count >=2:
                            break
                        if q == 'K':
                            outer_break = True # No need to check for other directions
                            if count == 0:
                                checks[0]+=1
                                checks.append(squareslist)
                            else:
                                pins[pinned_square]=squareslist
                            break
                        squareslist.append(j)
                    if outer_break:
                        break             
            else: #Opponents Crawlers
                for d in DirectionsDict[flipsquare][piece.upper()]:
                    if piece == 'p' and (d==N or d==N+N):
                        continue
                    if piece == 'p' and not downwards:
                        if d == 7:
                            d = -9
                        if d == 9:
                            d = -7
                    jflip = flipsquare + d
                    j = flipsquarelist[jflip]
                    q = self.board[j] # What is on destination square
                    if q == 'K':
                        checks[0]+=1
                        checks.append([square])
        return [checks, pins]
    def own_attacked_squares(self): # Getting the squares where our pieces are being attacked
        attacked_squares = []
        for square, piece in enumerate(self.board):
            if piece not in 'prqnbk': # We check for all opponents pieces 
                continue 
            downwards = False
            flipsquare = square # flipsquare is the corresponding square in the first quarterboard
            flipsquarelist = list(range(64))
            flipboard = self.board
            if any(square in range(start, start+4) for start in [4, 12, 20, 28]): 
                # h1 quadrant
                flipboard = [square for row in [self.board[i:i+8][::-1] for i in range(0, 64, 8)] for square in row] 
                flipsquarelist = [square for row in [list(range(64))[i:i+8][::-1] for i in range(0, 64, 8)] for square in row]
                flipsquare = flipsquarelist[square]                 
            elif any(square in range(start, start+4) for start in [36, 44, 52, 60]): # h8 quadrant
                # Quadrant h8
                downwards = True
                flipboard = list(reversed(self.board)) # Flip the board pieces horizontally
                flipsquarelist = list(reversed(range(64)))# correct
                flipsquare = flipsquarelist[square]
            elif any(square in range(start, start+4) for start in [32, 40, 48, 56]): # a8 quadrant
                # Quadrant a8
                downwards = True
                flipboard = [square for row in [self.board[i:i+8] for i in range(0, 64, 8)][::-1] for square in row]
                flipsquarelist = [square for row in [list(range(64))[::-1][i:i+8][::-1] for i in range(0, 64, 8)] for square in row]
                flipsquare = flipsquarelist[square]
            if piece in 'rqb': # Opponents sliders
                for d in DirectionsDict[flipsquare][piece.upper()].keys():
                    for k in range(DirectionsDict[flipsquare][piece.upper()][d]):
                        jflip = flipsquare+(k+1)*d
                        j = flipsquarelist[jflip]
                        q = flipboard[jflip] # What is on destination square
                        if q.islower(): # Own piece is blocking
                            break
                        if q.isupper() and q!= 'P':
                            attacked_squares.append(j)          
            else: #Opponents Crawlers
                for d in DirectionsDict[flipsquare][piece.upper()]:
                    d1 = d
                    if piece == 'p' and not downwards:
                        if d == 7:
                            d1 = -9
                        if d == 9:
                            d1 = -7
                        if d == 8:
                            continue
                    jflip = flipsquare + d1
                    j = flipsquarelist[jflip]
                    q = self.board[j] # What is on destination square
                    if q.isupper() and q!='P':
                        attacked_squares.append(j)
        return attacked_squares
    
    def out_of_attack_moves(self):
        score =0
        checks_pins = self.checks_pins()
        checks = checks_pins[0]
        pins = checks_pins[1]
        for square, piece in enumerate(self.board):
            attacked_squares = self.own_attacked_squares()
            # MAYBE INSTEAD OF CREATING FLIPBOARD AND FLIPSQUARE WE CAN CREATE A FOR LOOP, RUNNING THROUGH
            # THE DIRECTIONS ON EACH OF THE 4 BY 4 QUARTERBOARDS, I WILL CHECK WHEN FINISHED IF THIS IS MORE
            # EFFICIENT(MAYBE BECAUSE LESS MEMORY USAGE?).
            if not piece.isupper(): # We only check for white pieces(rotation will allow then to check for black)
                continue
            if square not in attacked_squares and checks[0]==0:
                continue
            downwards = False
            flipsquare = square # flipsquare is the corresponding square in the first quarterboard
            flipsquarelist = list(range(64))
            flipboard = self.board
            if any(square in range(start, start+4) for start in [4, 12, 20, 28]): 
                # h1 quadrant
                flipboard = [square for row in [self.board[i:i+8][::-1] for i in range(0, 64, 8)] for square in row] 
                flipsquarelist = [square for row in [list(range(64))[i:i+8][::-1] for i in range(0, 64, 8)] for square in row]
                flipsquare = flipsquarelist[square]                 
            elif any(square in range(start, start+4) for start in [36, 44, 52, 60]): # h8 quadrant
                # Quadrant h8
                downwards = True
                flipboard = list(reversed(self.board)) # Flip the board pieces horizontally
                flipsquarelist = list(reversed(range(64)))# correct
                flipsquare = flipsquarelist[square]
            elif any(square in range(start, start+4) for start in [32, 40, 48, 56]): # a8 quadrant
                # Quadrant a8
                downwards = True
                flipboard = [square for row in [self.board[i:i+8] for i in range(0, 64, 8)][::-1] for square in row]
                flipsquarelist = [square for row in [list(range(64))[::-1][i:i+8][::-1] for i in range(0, 64, 8)] for square in row]
                flipsquare = flipsquarelist[square]
            if piece in 'RQB' and checks[0]==0: # Sliders without checks
                for d in DirectionsDict[flipsquare][piece].keys():
                    for k in range(DirectionsDict[flipsquare][piece][d]):
                        jflip = flipsquare+(k+1)*d
                        j = flipsquarelist[jflip]
                        if square in pins.keys(): # If piece is pinned we can only move through pin
                            if j not in pins[square]:
                                break
                        q = flipboard[jflip] # What is on destination square
                        # Castling, by sliding the rook next to the king 
                        if q.isupper():
                            break # we cant move if our own piece is blocking
                        yield Move(square, j, "", score)
                        # Captures prevent more moves in that direction
                        if q.islower(): # Capture prevent more moves in that direction
                            break 
            elif piece in 'NK' and checks[0]==0: # Crawlers without checks
                for d in DirectionsDict[flipsquare][piece]:
                    jflip = flipsquare + d
                    j = flipsquarelist[jflip]
                    if square in pins.keys(): # If piece is pinned we can only move through pin
                        if j not in pins[square]:
                            continue
                    if piece == 'K': # Making sure we dont move the king in check
                        if self.null_move(Move(square, j, '', score)).rotate().checks_pins()[0][0] != 0:
                            continue
                    q = self.board[j] # What is on destination square
                    if q.isupper():
                        continue # we cant move if our own piece is blocking
                    yield Move(square, j, "", score)
                        
            elif piece in 'QRB' and checks[0] == 1: # In 1 check, Sliders can be captured, blocked or move king
                if square in pins.keys(): # If check, pinned pieces cant move
                    continue
                for d in DirectionsDict[flipsquare][piece].keys():
                    for k in range(DirectionsDict[flipsquare][piece][d]):
                        jflip = flipsquare+(k+1)*d
                        j = flipsquarelist[jflip]
                        q = flipboard[jflip] # What is on destination square
                        if q.isupper(): # we cant move more in that direction
                            break
                        if j not in checks[1]:
                            continue # We don't interfere with check
                        yield Move(square, j, '', score)
            elif piece in 'NPK' and checks[0] == 1: # In 1 check, Only capture or move king
                if square in pins.keys(): # If check, pinned pieces cant move
                    continue
                for d in DirectionsDict[flipsquare][piece]:
                    if piece == 'P' and downwards:
                        if d == 7:
                            d = -9
                        if d == 9:
                            d = -7
                        if d == 8:
                            d = -8
                    jflip = flipsquare + d
                    j = flipsquarelist[jflip]
                    if j not in checks[1] and piece != 'K':
                        continue
                    if piece == 'K': # Making sure we dont move the king in check
                        if self.null_move(Move(square, j, '', score)).rotate().checks_pins()[0][0] != 0:
                            continue
                    q = self.board[j] # What is on destination square
                    if q.isupper():
                        continue
                    if piece == "P": # Pawn move, double move and capture
                        if d in (N, N+N) and q != "0": # can't move
                            continue 
                        if d in (N+N,-N-N) and (square > 15 or flipboard[flipsquare + N] != "0"): # can't move twice
                            continue
                        if (d in (N+W, N+E, -N-W, -N-E) and (q == "0" and self.psquare != j)): 
                            continue # Can't capture or en passant 
                        if 56 <= j: # If we move to the last row, we can promote
                            for prom in "NBRQ":
                                yield Move(square, j, prom, score)
                                continue
                    yield Move(square,j, '', score)
            else: # There are multiple checks so we may only move king
                if piece != 'K':
                    continue
                for d in DirectionsDict[flipsquare]['K']:
                    jflip = flipsquare + d
                    j = flipsquarelist[jflip]
                    if piece == 'K': # Making sure we dont move the king in check
                        if self.null_move(Move(square, j, '', score)).rotate().checks_pins()[0][0] != 0:
                            continue
                    q = self.board[j]
                    if q.isupper():
                        continue
                    yield Move(square, j, '', score)
    
    def gen_moves(self):
        # The gen_moves method generates all legal moves for the current position, and returns 
        # a generator that yields Move objects. It iterates through each of the player's pieces 
        # on the board and, for each piece, iterates through each possible 'ray' of moves, as 
        # defined in the directions map. The rays are broken by captures or immediately in the 
        # case of crawlers such as knights.
        checks_pins = self.checks_pins()
        checks = checks_pins[0]
        pins = checks_pins[1]
        for square, piece in enumerate(self.board):
            # MAYBE INSTEAD OF CREATING FLIPBOARD AND FLIPSQUARE WE CAN CREATE A FOR LOOP, RUNNING THROUGH
            # THE DIRECTIONS ON EACH OF THE 4 BY 4 QUARTERBOARDS, I WILL CHECK WHEN FINISHED IF THIS IS MORE
            # EFFICIENT(MAYBE BECAUSE LESS MEMORY USAGE?).
            if not piece.isupper(): # We only check for white pieces(rotation will allow then to check for black)
                continue
            downwards = False
            flipsquare = square # flipsquare is the corresponding square in the first quarterboard
            flipsquarelist = list(range(64))
            flipboard = self.board
            if any(square in range(start, start+4) for start in [4, 12, 20, 28]): 
                # h1 quadrant
                flipboard = [square for row in [self.board[i:i+8][::-1] for i in range(0, 64, 8)] for square in row] 
                flipsquarelist = [square for row in [list(range(64))[i:i+8][::-1] for i in range(0, 64, 8)] for square in row]
                flipsquare = flipsquarelist[square]                 
            elif any(square in range(start, start+4) for start in [36, 44, 52, 60]): # h8 quadrant
                # Quadrant h8
                downwards = True
                flipboard = list(reversed(self.board)) # Flip the board pieces horizontally
                flipsquarelist = list(reversed(range(64)))# correct
                flipsquare = flipsquarelist[square]
            elif any(square in range(start, start+4) for start in [32, 40, 48, 56]): # a8 quadrant
                # Quadrant a8
                downwards = True
                flipboard = [square for row in [self.board[i:i+8] for i in range(0, 64, 8)][::-1] for square in row]
                flipsquarelist = [square for row in [list(range(64))[::-1][i:i+8][::-1] for i in range(0, 64, 8)] for square in row]
                flipsquare = flipsquarelist[square]
            if piece in 'RQB' and checks[0]==0: # Sliders without checks
                for d in DirectionsDict[flipsquare][piece].keys():
                    for k in range(DirectionsDict[flipsquare][piece][d]):
                        score = 0
                        jflip = flipsquare+(k+1)*d
                        j = flipsquarelist[jflip]
                        if square in pins.keys(): # If piece is pinned we can only move through pin
                            if j not in pins[square]:
                                break
                        q = flipboard[jflip] # What is on destination square
                        if q.islower():
                            score += 1 # capture gets a score
                            if q != 'p':
                                score +=1
                        if self.null_move(Move(square, j, '',0)).checks_pins()[0][0]!=0:
                            score +=3 # Checks get a score
                        
                        # Castling, by sliding the rook next to the king 
                        if square == 0 and self.wc[1] and q=='K':
                            position2 = self
                            if self.null_move(Move(j+E, j, '', 0)).rotate().checks_pins()[0][0]!=0 or position2.null_move(Move(j+E, j+W, '', 0)).rotate().checks_pins()[0][0]!=0:
                                break
                            yield Move(j + E, j + W, "", score) # castling is represented by the king move (makes 
                                                 # move method easier)
                            break
                        if square == 7 and self.wc[0] and q=='K':
                            position2 = self
                            if self.null_move(Move(j+W, j, '', 0)).rotate().checks_pins()[0][0]!=0 or position2.null_move(Move(j+W, j+E, '', 0)).rotate().checks_pins()[0][0]!=0:
                                break
                            yield Move(j + W, j + E, "", score)
                            break
                        if q.isupper():
                            break # we cant move if our own piece is blocking
                        yield Move(square, j, "", score)
                        # Captures prevent more moves in that direction
                        if q.islower(): # Capture prevent more moves in that direction
                            break
            elif piece in 'NKP' and checks[0]==0: # Crawlers without checks
                for d in DirectionsDict[flipsquare][piece]:
                    score = 0
                    if piece == 'P' and downwards:
                        if d == 7:
                            d = -9
                        if d == 9:
                            d = -7
                        if d == 8:
                            d = -8
                    jflip = flipsquare + d
                    j = flipsquarelist[jflip]
                    if square in pins.keys(): # If piece is pinned we can only move through pin
                        if j not in pins[square]:
                            continue
                    if piece == 'K': # Making sure we dont move the king in check
                        if self.null_move(Move(square, j, '', score)).rotate().checks_pins()[0][0] != 0:
                            continue
                    q = self.board[j] # What is on destination square
                    if q.isupper():
                        continue # we cant move if our own piece is blocking
                    if piece == "P": # Pawn move, double move and capture
                        if d in (N, N+N,-N,-N-N) and q != "0": # can't move
                            continue 
                        if d in (N+N,-N-N) and (square > 15 or flipboard[flipsquare + N] != "0"): # can't move twice
                            continue
                        if (d in (7, -7, 9, -9) and (q == "0" and self.psquare != j)): 
                            continue # Can't capture or en passant 
                        if 56 <= j: # If we move to the last row, we can promote
                            for prom in "NBRQ":
                                yield Move(square, j, prom, score)
                            continue
                        if self.psquare ==j and d in (N+W, N+E,-N-W,-N-E):# Sometimes we cant enpassant because otherwise we are in check, however the pawn is not pinned
                            if self.null_move(Move(square, j, '', 0)).rotate().checks_pins()[0][0]!=0:
                                continue
                    
                    if q.islower() or j==self.psquare:
                        score += 1
                        if q != 'p':
                            score += 1 
                    
                    if self.null_move(Move(square, j, '',0)).checks_pins()[0][0]!=0:
                        score +=3 # Checks get a score
                    yield Move(square, j, "", score)
                    
            elif piece in 'QRB' and checks[0] == 1: # In 1 check, Sliders can be captured, blocked or move king
                if square in pins.keys(): # If check, pinned pieces cant move
                    continue
                for d in DirectionsDict[flipsquare][piece].keys():
                    for k in range(DirectionsDict[flipsquare][piece][d]):
                        score = 0
                        jflip = flipsquare+(k+1)*d
                        j = flipsquarelist[jflip]
                        q = flipboard[jflip] # What is on destination square
                        if q.isupper(): # we cant move more in that direction
                            break
                        if j not in checks[1]:
                            continue # We don't interfere with check
                        if q.islower():
                            score += 1
                            if q != 'p':
                                score += 1
                        yield Move(square, j, '', score)
            elif piece in 'NPK' and checks[0] == 1: # In 1 check, Only capture or move king
                if square in pins.keys(): # If check, pinned pieces cant move
                    continue
                for d in DirectionsDict[flipsquare][piece]:
                    score = 0
                    if piece == 'P' and downwards:
                        if d == 7:
                            d = -9
                        if d == 9:
                            d = -7
                        if d == 8:
                            d = -8
                    jflip = flipsquare + d
                    j = flipsquarelist[jflip]
                    if j not in checks[1] and piece != 'K' and piece != 'P':
                        continue # King can move out of check and en passant can capture pawn giving check without moving to pawns square
                    if piece == 'P' and j not in checks[1] and j !=self.psquare:
                        continue # unless en passant, if we cant capture piece directly or block we cant move pawn
                    if piece =='P' and j==self.psquare:
                        if self.null_move(Move(square, j, '', score)).rotate().checks_pins()[0][0] != 0:
                            print('MAL')
                            continue # If we capture en passant we see if we are in check after capturing
                    if piece == 'K': # Making sure we dont move the king in check
                        if self.null_move(Move(square, j, '', score)).rotate().checks_pins()[0][0] != 0:
                            continue
                    q = self.board[j] # What is on destination square
                    if q.isupper():
                        continue
                    if q.islower():
                        score += 1
                        if q != 'p':
                            score += 1
                    if piece == "P": # Pawn move, double move and capture
                        if d in (N, N+N) and q != "0": # can't move
                            continue 
                        if d in (N+N,-N-N) and (square > 15 or flipboard[flipsquare + N] != "0"): # can't move twice
                            continue
                        if (d in (N+W, N+E, -N-W, -N-E) and (q == "0" and self.psquare != j)):
                            continue # Can't capture or en passant 
                        if 56 <= j: # If we move to the last row, we can promote
                            for prom in "NBRQ":
                                yield Move(square, j, prom, score)
                            continue
                    yield Move(square,j, '', score)
            else: # There are multiple checks so we may only move king
                if piece != 'K':
                    continue
                for d in DirectionsDict[flipsquare]['K']:
                    score = 0
                    jflip = flipsquare + d
                    j = flipsquarelist[jflip]
                    if piece == 'K': # Making sure we dont move the king in check
                        if self.null_move(Move(square, j, '', score)).rotate().checks_pins()[0][0] != 0:
                            continue
                    q = self.board[j]
                    if q.isupper():
                        continue
                    if q.islower():
                        score += 1
                        if q != 'p':
                            score += 1
                    yield Move(square, j, '', score)
    def own_quiet(self): 
        checks_pins = self.checks_pins()
        checks = checks_pins[0]
        pins = checks_pins[1]
        if checks[0]!= 0: # If there is a check, position isn't quiet
            return False
        for square, piece in enumerate(self.board):
            if not piece.isupper(): # We only check for our own pieces
                continue
            downwards = False
            flipsquare = square # flipsquare is the corresponding square in the first quarterboard
            flipsquarelist = list(range(64))
            flipboard = self.board
            if any(square in range(start, start+4) for start in [4, 12, 20, 28]): 
                # Quadrant h1
                flipboard = [square for row in [self.board[i:i+8][::-1] for i in range(0, 64, 8)] for square in row] 
                flipsquarelist = [square for row in [list(range(64))[i:i+8][::-1] for i in range(0, 64, 8)] for square in row]
                flipsquare = flipsquarelist[square]                 
            elif any(square in range(start, start+4) for start in [36, 44, 52, 60]):
                # Quadrant h8
                downwards = True
                flipboard = list(reversed(self.board)) # Flip the board pieces horizontally
                flipsquarelist = list(reversed(range(64)))# correct
                flipsquare = flipsquarelist[square]
            elif any(square in range(start, start+4) for start in [32, 40, 48, 56]):
                # Quadrant a8
                downwards = True
                flipboard = [square for row in [self.board[i:i+8] for i in range(0, 64, 8)][::-1] for square in row]
                flipsquarelist = [square for row in [list(range(64))[::-1][i:i+8][::-1] for i in range(0, 64, 8)] for square in row]
                flipsquare = flipsquarelist[square]
            if piece in 'RQB': # Our sliders
                for d in DirectionsDict[flipsquare][piece].keys():
                    for k in range(DirectionsDict[flipsquare][piece][d]):
                        jflip = flipsquare+(k+1)*d
                        j = flipsquarelist[jflip]
                        if square in pins.keys(): # If piece is pinned we can only move through pin
                            if j not in pins[square]:
                                break
                        q = flipboard[jflip] # What is on destination square
                        if q.isupper(): # We cant move more in that direction
                            break
                        if q.islower() and q!='p': # we can capture a piece
                            return False
                        if q=='p':
                            break
                        
            if piece in 'KNP': # Our crawlers
                for d in DirectionsDict[flipsquare][piece]:
                    if piece == 'P' and (d == N or d == N+N):
                        continue # We cant capture with pawns moving upwards
                    d1 = d
                    if piece == 'P' and downwards:
                        if d == 7:
                            d1 = -9
                        if d == 9:
                            d1 = -7
                    jflip = flipsquare + d1
                    j = flipsquarelist[jflip]
                    if square in pins.keys(): # If piece is pinned we can only move through pin
                            if j not in pins[square]:
                                break
                    q = flipboard[jflip] 
                    if q.islower() and q!='p':
                        return False
        return True
                    
    def rotate(self): 
        # Switch the roles of black and white. It also swaps the case of all the pieces and preserves 
        # en passant unless it is a null move. White's castling rights become black's and viceversa.
        return Position([x.swapcase() for x in self.board[::-1]], self.psquare, self.bc, self.wc, self.history, self.turn)
    
    def null_move(self, move): 
        # Null move doesnt change actual position, it only creates a new position, where the only
       # thing changing is the board, it makes a move and changes players perspective 
        position_copy = copy.deepcopy(self)
        i, j, prom, capt = move # prom is the letter representing the piece we are promoting to 
        piece, q, board = position_copy.board[i], position_copy.board[j], position_copy.board # piece and destination piece/space
        wc, bc = position_copy.wc, position_copy.bc
        put = lambda board, i, p: board[:i] + [p] + board[i+1:]
        newpsquare, newwc, newbc = -1, position_copy.wc, position_copy.bc 
        # Actual move
        newboard = put(position_copy.board, j, position_copy.board[i])
        newboard = put(newboard, i, "0")
        # If we move the rook or capture the opponent's the castling rights are lost
        if i == 0: wc = (False, wc[1])
        if i == 7: wc = (wc[0], False)
        if j == 56: bc = (bc[0], False)
        if j == 63: bc = (False, bc[1])
        # Castling 
        if piece == "K":
            wc = (False, False) # if king moves then player can't castle
            if abs(j - i) == 2: # if castling... ¡¡¡ARREGLAR ESTO!!!
                newboard = put(newboard, 0 if j < i else 7, "0")
                newboard = put(newboard, 2 if j < i else 4, "R")
        # Pawn promotion, double move and en passant capture
        if piece == "P":
            if 56 <= j:
                newboard = put(newboard, j, prom) # changing the pawn to the promoting piece
            if j - i == N + N: # If we move pawn two squares, the enpassant square becomes...
                newpsquare = list(range(64))[i + N]   
            if j == position_copy.psquare: # If we capture en passant
                newboard = put(newboard, j + S, "0")
        # We rotate the returned position, so it's ready for the next player
        # Making a move will change the actual position object
        # This will later make code more memory efficient
        newboard = [x.swapcase() for x in newboard[::-1]]
        return Position(newboard, newpsquare, newwc, newbc, position_copy.history, position_copy.turn)
    
    def opponent_quiet(self):
        opponent_position = self
        return  opponent_position.rotate().own_quiet()
        
    def move(self, move):
        # The move method takes a Move object as an argument and returns a new Position 
        # object representing the state of the game after the move has been made. It first 
        # checks if the move is castling, pawn promotion, or en passant, and updates the 
        # board accordingly. It then updates the castling rights and en passant square if 
        # necessary, and rotates the board to return a new Position object.
        i, j, prom, capt = move # prom is the letter representing the piece we are promoting to 
        piece, q, board = self.board[i], self.board[j], self.board # piece and destination piece/space
        wc, bc = self.wc, self.bc
        put = lambda board, i, p: board[:i] + [p] + board[i+1:]
        newpsquare, newwc, newbc = -1, self.wc, self.bc 
        # Actual move
        newboard = put(self.board, j, self.board[i])
        newboard = put(newboard, i, "0")
        # If we move the rook or capture the opponent's the castling rights are lost
        if i == 0: wc = (False, wc[1])
        if i == 7: wc = (wc[0], False)
        if j == 56: bc = (bc[0], False)
        if j == 63: bc = (False, bc[1])
        # Castling 
        if piece == "K":
            wc = (False, False) # if king moves then player can't castle
            if abs(j - i) == 2: # if castling... ¡¡¡ARREGLAR ESTO!!!
                newboard = put(newboard, 0 if j < i else 7, "0")
                newboard = put(newboard, 2 if j < i else 4, "R")
        # Pawn promotion, double move and en passant capture
        if piece == "P":
            if 56 <= j:
                newboard = put(newboard, j, prom) # changing the pawn to the promoting piece
            if j - i == N + N: # If we move pawn two squares, the enpassant square becomes...
                newpsquare = list(reversed(range(64)))[i + N]   
            if j == self.psquare: # If we capture en passant
                newboard = put(newboard, j + S, "0")
        # We rotate the returned position, so it's ready for the next player
        if self.turn == True: # If white has moved
            self.history.append(''.join(newboard)) # Making a move will add the new board to history
        if self.turn == False: # If black has moved we add board to history from white's perspective
            self.history.append(''.join([x.swapcase() for x in newboard[::-1]]))
        self.turn = not self.turn # making a move changes turn
        # Making a move will change the actual position object
        # This will later make code more memory efficient
        self.board, self.psquare, self.wc, self.bc = [x.swapcase() for x in newboard[::-1]], newpsquare, newwc, newbc
        return self
    
    def ordered_moves(self): # We order legal moves in terms of score
        moves = self.gen_moves()
        return sorted(moves, key=lambda move: move.score, reverse=True) 
    
    def three_fold(self): # The engine will see repeating a position as evaluation 0
        counted = Counter(self.history)
        if any(count >= 2 for count in counted.values()):
            return True # 3-fold repetition
        else: 
            return False
    def pop(self): # ¡¡¡ADD ALSO WC BC PSQUARE(AND FOR HISTORY TOO)!!!
        self.history.pop(-1) # Delete current board from history
        self.turn = not self.turn # We change turns
        board = list(self.history[-1])
        if self.turn == True: # If after going back it is white's turn
            self.board = board # Current board -> last board
        if self.turn == False:
            self.board = [x.swapcase() for x in board[::-1]] # we want self.board to be from players move perspective
        return self
    def quiet(self):
        if self.own_quiet() and self.opponent_quiet():
            return True
        else:
            return False
    

In [733]:
board = [ 'R', 'N', 'B', 'Q', 'K', 'B', 'N', 'R',          
         'P', 'P', 'P', 'P', 'P', 'P', 'P', 'P',          
         '0', '0', '0', '0', '0', '0', '0', '0',          
         '0', '0', '0', '0', '0', '0', '0', '0',       
         '0', '0', '0', '0', '0', '0', '0', '0',          
         '0', '0', '0', '0', '0', '0', '0', '0',          
         'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p',          
         'r', 'n', 'b', 'q', 'k', 'b', 'n', 'r' ]
print(Position(board,-1,[True, True], [True, True], [], True).move(Move(1,16,'',0)).move(Move(1,16,'',0)).move(Move(16,1,'',0)).move(Move(16,1,'',0)).move(Move(1,16,'',0)).move(Move(1,16,'',0)).move(Move(16,1,'',0)).move(Move(16,1,'',0)).three_fold())


True


In [737]:
p2 = fen_to_position('5r1b/2R1R3/P4r2/2p2Nkp/2b3pN/6P1/4PP2/6K1 w - - 0 1')
pretty_print(p2.board)
p2.checks_pins()


.....♜.♝
..♖.♖...
♙....♜..
..♟..♘♚♟
..♝...♟♘
......♙.
....♙♙..
......♔.


[[0], {}]

In [738]:
p2.move(Move(13, 29, '', 0))
pretty_print(p2.board)
print(list(p2.gen_moves()))
print(p2.psquare)
print(p2.checks_pins())

.♚......
...♟....
.♟......
♞♙♟..♗..
♙♔♞..♙..
..♖....♟
...♜.♜..
♗.♖.....
[Move(i=18, j=34, prom='', score=1), Move(i=33, j=42, prom='', score=0)]
42
[[1, [34]], {}]


## Simple Position Evaluation

In [365]:
pst = {
    'P': (   0,   0,   0,   0,   0,   0,   0,   0,
            78,  83,  86,  73, 102,  82,  85,  90,
             7,  29,  21,  44,  40,  31,  44,   7,
           -17,  16,  -2,  15,  14,   0,  15, -13,
           -26,   3,  10,   9,   6,   1,   0, -23,
           -22,   9,   5, -11, -10,  -2,   3, -19,
           -31,   8,  -7, -37, -36, -14,   3, -31,
             0,   0,   0,   0,   0,   0,   0,   0),
    'N': ( -66, -53, -75, -75, -10, -55, -58, -70,
            -3,  -6, 100, -36,   4,  62,  -4, -14,
            10,  67,   1,  74,  73,  27,  62,  -2,
            24,  24,  45,  37,  33,  41,  25,  17,
            -1,   5,  31,  21,  22,  35,   2,   0,
           -18,  10,  13,  22,  18,  15,  11, -14,
           -23, -15,   2,   0,   2,   0, -23, -20,
           -74, -23, -26, -24, -19, -35, -22, -69),
    'B': ( -59, -78, -82, -76, -23,-107, -37, -50,
           -11,  20,  35, -42, -39,  31,   2, -22,
            -9,  39, -32,  41,  52, -10,  28, -14,
            25,  17,  20,  34,  26,  25,  15,  10,
            13,  10,  17,  23,  17,  16,   0,   7,
            14,  25,  24,  15,   8,  25,  20,  15,
            19,  20,  11,   6,   7,   6,  20,  16,
            -7,   2, -15, -12, -14, -15, -10, -10),
    'R': (  35,  29,  33,   4,  37,  33,  56,  50,
            55,  29,  56,  67,  55,  62,  34,  60,
            19,  35,  28,  33,  45,  27,  25,  15,
             0,   5,  16,  13,  18,  -4,  -9,  -6,
           -28, -35, -16, -21, -13, -29, -46, -30,
           -42, -28, -42, -25, -25, -35, -26, -46,
           -53, -38, -31, -26, -29, -43, -44, -53,
           -30, -24, -18,   5,  -2, -18, -31, -32),
    'Q': (   6,   1,  -8,-104,  69,  24,  88,  26,
            14,  32,  60, -10,  20,  76,  57,  24,
            -2,  43,  32,  60,  72,  63,  43,   2,
             1, -16,  22,  17,  25,  20, -13,  -6,
           -14, -15,  -2,  -5,  -1, -10, -20, -22,
           -30,  -6, -13, -11, -16, -11, -16, -27,
           -36, -18,   0, -19, -15, -15, -21, -38,
           -39, -30, -31, -13, -31, -36, -34, -42),
    'K': (   4,  54,  47, -99, -99,  60,  83, -62,
           -32,  10,  55,  56,  56,  55,  10,   3,
           -62,  12, -57,  44, -67,  28,  37, -31,
           -55,  50,  11,  -4, -19,  13,   0, -49,
           -55, -43, -52, -28, -51, -47,  -8, -50,
           -47, -42, -43, -79, -64, -32, -29, -32,
            -4,   3, -14, -50, -57, -18,  13,   4,
            17,  30,  -3, -14,   6,  -1,  40,  18),
}

In [625]:
def evaluate_position_from_fen(fen): # Simple position evaluation to check engine algorithm
    board = chess.Board(fen)
    material_value = {
        "P": 100, "N": 300, "B": 300, "R": 500, "Q": 900,
        "p": -100, "n": -300, "b": -300, "r": -500, "q": -900
    }
    total_material = 0
    for piece in fen.split()[0]:
        if piece in material_value:
            total_material += material_value[piece][square]
    return total_material

def evaluate_position_from_board(board): # Simple position evaluation to check engine algorithm
    material_value = {
        "P": 100, "N": 300, "B": 300, "R": 500, "Q": 900,
        "p": -100, "n": -300, "b": -300, "r": -500, "q": -900
    }
    total_material = 0
    for square, piece in enumerate(board):
        if piece in material_value:
            total_material += material_value[piece]
            # total_material += pst[piece.upper()][square]
    return total_material

In [1011]:
import time

start_time = time.time()

# Example FEN string
fen = "r2qkbnr/1bp2ppp/1pnp4/pB2p3/3PP3/2N2N2/PPP2PPP/R1BQ1RK1 w kq - 2 7"

depth = 5
alpha = -np.inf
beta = np.inf
maximize = True
best_move = None
best_value = -np.inf

board = chess.Board(fen)
legal_moves = list(board.legal_moves)

for move in legal_moves:
    board.push(move)
    new_fen = board.fen()
    x = alpha_beta_from_fen(new_fen, depth - 1, alpha, beta, not maximize, evaluate_position)
    board.pop()

    if x[0] > best_value:
        best_move = move
        best_value = x[0]
        variation = [move] + x[1]

print("Best move: ", best_move)
print("Value: ", best_value)
end_time = time.time()
elapsed_time = end_time - start_time

print("Time taken:", elapsed_time, "seconds")
print('Variation:', variation)


NameError: name 'evaluate_position' is not defined

In [22]:
def fen_to_position(fen):
    board = ['0']*64
    fen_parts = fen.split()
    fen_board = fen_parts[0].split('/')
    for i in range(len(fen_board)):
        fen_row = fen_board[i]
        j = 0
        for c in fen_row:
            if c.isdigit():
                j += int(c)
            else:
                board[(7-i)*8+j] = c
                j += 1
    
    en_passant = chess.SQUARE_NAMES.index(fen_parts[3]) if fen_parts[3] != '-' else -1
    
    white_castling_rights = [False]*2
    if 'K' in fen_parts[2]:
        white_castling_rights[0] = True
    if 'Q' in fen_parts[2]:
        white_castling_rights[1] = True
    black_castling_rights = [False]*2
    if 'k' in fen_parts[2]:
        black_castling_rights[0] = True
    if 'q' in fen_parts[2]:
        black_castling_rights[1] = True
    
    turn = fen_parts[1] == 'w'
    
    return Position(board, en_passant, white_castling_rights, black_castling_rights, [], turn)


In [23]:
import chess

def position_to_fen(position):
    board = chess.Board()
    board.set_board_fen(position.board_fen())
    fen_parts = [
        board.board_fen(),
        'w' if position.turn else 'b',
        ''.join(['K' if position.castling_rights[0] else '',
                 'Q' if position.castling_rights[1] else '',
                 'k' if position.castling_rights[2] else '',
                 'q' if position.castling_rights[3] else '']),
        chess.SQUARE_NAMES[position.en_passant] if position.en_passant != -1 else '-',
        str(position.halfmove_clock),
        str(position.fullmove_number)
    ]
    return ' '.join(fen_parts)

### The Simple Algorithms

I am creating two simple algorithms, one which runs with the python chess library (alpha_beta_from_fen) and the other with my move generator. My move generator appears to make the algorithm 2 times faster.

In [123]:
import chess
def alpha_beta_from_fen(fen, depth, alpha, beta, maximize, evaluate_position): 
    # maximize is a boolean variable (True if we want to maximize the evaluation (we are white))
    # alpha is the current best evaluation for white, it will start at -infinity
    # beta is the current best evaluation for black, it will start at +infinity
    
    board = chess.Board(fen = fen)
    legal_moves = list(board.legal_moves)
    
    if depth == 0:
        return evaluate_position(fen), []
    
    best_move = None
    
    if maximize:
        value = -1000
        for move in legal_moves:
            board.push(move)
            child_value = alpha_beta_from_fen(board.fen(), depth - 1, alpha, beta, False, evaluate_position)[0]
            if child_value > value:
                value = child_value
                best_move = move
            board.pop() # Unmake the last move
            alpha = max(alpha, value)
            if alpha >= beta:
                break
    else:
        value = 1000
        for move in legal_moves:
            board.push(move)
            child_value = alpha_beta_from_fen(board.fen(), depth - 1, alpha, beta, True, evaluate_position)[0]
            if child_value < value:
                value = child_value
                best_move = move
            board.pop() 
            beta = min(beta, value)
            if beta <= alpha:
                break
    
    return value, best_move

In [124]:
def engine_fen(fen, is_maximizing_player, is_our_move, depth): # ¡¡¡ FIX AND ADD VARIATIONS!!!
    # is_maximizing_player is set to True if we are white
    # alpha is the current best evaluation for white, it will start at -infinity
    # beta is the current best evaluation for black, it will start at +infinity
    start_time = time.time()
    best_move = None
    board = chess.Board(fen)
    legal_moves = list(board.legal_moves)

    if is_our_move:
        if is_maximizing_player:
            alpha = -np.inf
            beta = np.inf
            best_value = -np.inf
        else:
            alpha = -np.inf
            beta = np.inf
            best_value = np.inf
        
        for move in legal_moves:
            board.push(move)
            new_fen = board.fen()
            value = alpha_beta_from_fen(new_fen, depth - 1, alpha, beta, not is_maximizing_player, evaluate_position_from_fen)[0]
            board.pop()

            if is_maximizing_player and value > best_value:
                best_move = move
                best_value = value
            elif is_maximizing_player==False and value < best_value:
                best_move = move
                best_value = value
        end_time = time.time()
        elapsed_time = end_time - start_time
        return ["Time taken:", elapsed_time, "seconds", "Best move: ", best_move, "Evaluation: ", best_value]
    else: # Now we make the engine think while it isn't it's turn
        Dict = {}
        for move in legal_moves:
            board.push(move)
            new_fen = board.fen()
            best_move = None
            Dict[move]=(engine(fen, is_maximizing_player, is_our_move, depth+1)[4], 
                        engine(fen, is_maximizing_player, is_our_move, depth+1)[6])
        return(Dict) # This is a dictionary of the form {move: best response, evaluation}
        

In [125]:
engine_fen('r2qkbnr/1bp2ppp/1pnp4/pB2p3/3PP3/2N2N2/PPP2PPP/R1BQ1RK1 w kq - 2 7', True, True, 5)

['Time taken:',
 175.86417317390442,
 'seconds',
 'Best move: ',
 Move.from_uci('d4e5'),
 'Evaluation: ',
 4]

This is the best move, in a position where we are black and it is our move. For a search of depth 5, it takes 154 seconds which should be improved, it is finding the best move in a simple capture. Note that we are using a very simple evaluation function which will be faster than the true one.

In [140]:
def alpha_beta_from_position(position, depth, alpha, beta, maximize):
    # maximize is a boolean variable (True if we want to maximize the evaluation (we are moving))
    # alpha is the current best evaluation for us, it will start at -infinity
    # beta is the current best evaluation for opponent, it will start at +infinity
    if depth == 0 and maximize:
        # If in the last move it is our turn, the position will be evaluated as if we are white, since we are 
        # the maximizing player, this is the correct sign
        return evaluate_position_from_board(position.board), 0
    if depth == 0 and not maximize:
        # If in the last move is the opponents turn, the position will be evaluated as if we are black, since
        # we are the maximizing player, we change the sign
        return -evaluate_position_from_board(position.board), 0 
    
    legal_moves = position.gen_moves()
    best_move = None
    
    if maximize: # If we have to move, we want to maximize. This is ensured by the above
        value = -10000 # This is the best evaluation for white in the child_values
        for move in legal_moves:
            position1 = position
            child_value = alpha_beta_from_position(position1.move(move), depth - 1, alpha, beta, False)[0]
            if child_value > value:
                value = child_value
                best_move = move
            alpha = max(alpha, value)
            if alpha >= beta:
                break # Because if our best move is better than the best in another set of moves that lead from
                      # a different move from opponent, then the opponent will choose the other move.
                
    else: # If opponent has to move
        value = 10000
        for move in legal_moves:
            position1 = position
            child_value = alpha_beta_from_position(position1.move(move), depth - 1, alpha, beta, True)[0]
            if child_value < value:
                value = child_value
                best_move = move
            beta = min(beta, value)
            if alpha >= beta: # Because if our opponents best move is better (less than) than the best 
                break         # in another set of moves that lead from a different move from us, 
                              # then we will choose the other move.
    
    return value, best_move 


In [141]:
import time
import numpy as np
def engine_position(position, is_our_move, depth): 
    # alpha is the current best evaluation for white, it will start at -1000
    # beta is the current best evaluation for black, it will start at +1000
    start_time = time.time()
    best_move = None
    legal_moves = position.gen_moves()

    if is_our_move:
        alpha = -10000 # Current best evaluation for us
        beta = 10000 # Current best evaluation for opponent
        best_value = -10000
        best_value, best_move = alpha_beta_from_position(position, depth, alpha, beta, True)
        end_time = time.time()
        elapsed_time = end_time - start_time
        return ["Time taken:", elapsed_time, "seconds", "Best move: ", best_move, "Evaluation: ", best_value]
    else: # Now we make the engine think while it isn't it's turn
        Dict = {}
        for move in legal_moves:
            new_position = position.move(move)
            best_move = None
            Dict[move]=(engine_position(new_position, is_maximizing_player, is_our_move, depth+1)[4], 
                        engine_position(new_position, is_maximizing_player, is_our_move, depth+1)[6])
        return(Dict) # This is a dictionary of the form {move: best response, evaluation}


In [142]:
engine_position(fen_to_position('r2qkbnr/1bp2ppp/1pnp4/pB2p3/3PP3/2N2N2/PPP2PPP/R1BQ1RK1 w kq - 2 7'), True, 5)


['Time taken:',
 28.686916828155518,
 'seconds',
 'Best move: ',
 Move(i=27, j=36, prom='', score=2),
 'Evaluation: ',
 5]

In [154]:
engine_position(fen_to_position('rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1'), True, True, 6)

['Time taken:',
 20.29952311515808,
 'seconds',
 'Best move: ',
 Move(i=12, j=20, prom='', capt='0'),
 'Evaluation: ',
 0]

### Improving the Algorithms 

There are several ways to improve the alpha-beta algorithm for chess:

* Iterative Deepening: This technique involves running the alpha-beta search multiple times with increasing depth limits. It allows the program to quickly find a good move at lower depths, and then continue searching deeper if there is time remaining.

* Killer moves: This are moves stored during the search when iterative deepening. If a move creates a cutoff during one step of iterative deepening (running alphabeta on a fixed depth), then it is stored and on the next step of iterative deepening it will be tried the first move.

* Transposition Tables: This technique involves storing previously computed evaluations in a hash table, so that the algorithm can avoid redundant evaluations. This can be particularly useful when searching the same position from different move orders.

* Move Ordering: This technique involves ordering the moves so that the most promising moves are searched first. This can lead to pruning more branches and searching deeper in the remaining branches. **DONE WITH CAPTURES AND CEHCKS**

* Quiescence Search: This technique involves extending the search depth for positions with captures and checks, as these are often critical in chess. This can help avoid the "horizon effect" where a position looks good at the current depth, but leads to a bad position in the next ply. **DONE**

* Null Move Pruning: This technique involves temporarily passing the turn to the opponent and then evaluating the position. If the evaluation is significantly worse than the current best score, then the move leading to that position is unlikely to be good, and the algorithm can skip searching that move.

* Futility Pruning: This technique involves pruning branches where the evaluation is already significantly worse than the current best score. This can reduce the search depth and speed up the algorithm.

In [24]:
Squares_Dict = ['a1', 'b1', 'c1', 'd1', 'e1', 'f1', 'g1', 'h1',
                'a2', 'b2', 'c2', 'd2', 'e2', 'f2', 'g2', 'h2',
                'a3', 'b3', 'c3', 'd3', 'e3', 'f3', 'g3', 'h3',
                'a4', 'b4', 'c4', 'd4', 'e4', 'f4', 'g4', 'h4',
                'a5', 'b5', 'c5', 'd5', 'e5', 'f5', 'g5', 'h5',
                'a6', 'b6', 'c6', 'd6', 'e6', 'f6', 'g6', 'h6',
                'a7', 'b7', 'c7', 'd7', 'e7', 'f7', 'g7', 'h7',
                'a8', 'b8', 'c8', 'd8', 'e8', 'f8', 'g8', 'h8']


In [708]:
import time
import numpy as np
class Engine:
    def alpha_beta( position, depth, alpha, beta, our_turn):
        ordered_moves = position.ordered_moves()
        if position.three_fold(): # Repetitions
            return 0, 0 
    
        if len(ordered_moves)==0 and position.checks_pins()[0][0]==0: # Stalemate
            return 0, 0
    
        if len(ordered_moves)==0 and our_turn: # Checkmate against us
            return -1003, 0
    
        if len(ordered_moves)==0 and not our_turn: # Checkmate against opponent
            return 1003, 0
    
        if depth <= 0 and position.quiet() and our_turn:
            return evaluate_position_from_board(position.board), 0
    
        if depth <= 0 and position.quiet() and not our_turn:
            return -evaluate_position_from_board(position.board), 0
        
        if depth<= 0 and not position.quiet(): # If depth is reached we continue with captures (not pawns) and
            ordered_moves = [move for move in ordered_moves if move.score >= 2] # checks until quiet position
        
        '''if len(ordered_moves)==0 and len(position.own_attacked_squares())>=2:
            print('strange case where king cannot take 1')
            ordered_moves = list(position.out_of_attack_moves())'''
        
        if len(ordered_moves)==0 and our_turn:
            return evaluate_position_from_board(position.board), 0
            
            
        if len(ordered_moves)==0 and not our_turn:
            return -evaluate_position_from_board(position.board), 0
            
        best_move = ordered_moves[0]
    
        if our_turn: # If we have to move, we want to maximize. This is ensured by the above
            value1 = -1000 # This is the best evaluation for white in the child_values
            for move in ordered_moves:
                position.move(move)
                child_value = Engine.alpha_beta(position, depth - 1, alpha, beta, False)[0]
                if child_value > value1:
                    value1 = child_value
                    best_move = move
                position.pop()
                if value1 >= beta:
                    break # Because if our best move is better than the best in another set of moves that lead from
                alpha = max(alpha, value1)      # a different move from opponent, then the opponent will choose the other move.
                
        else: # If opponent has to move
            value2 = 1001
            for move in ordered_moves:
                position.move(move)
                child_value = Engine.alpha_beta(position, depth - 1, alpha, beta, True)[0]
                if child_value < value2:
                    value2 = child_value
                    best_move = move
                position.pop()
                if alpha >= value2: # Because if our opponents best move is better (less than) than the best 
                    break         # in another set of moves that lead from a different move from us, 
                beta = min(beta, value2)     # then we will choose the other move.
        return (value1, best_move) if our_turn else (value2,best_move)
    
    def Search(position, is_our_move, depth): 
        # alpha is the current best evaluation for white, it will start at -1000
        # beta is the current best evaluation for black, it will start at +1000
        start_time = time.time()
        best_move = None
        if is_our_move:
            alpha = -10000 # Current best evaluation for us
            beta = 10000 # Current best evaluation for opponent
            best_value, best_move = Engine.alpha_beta(position, depth, alpha, beta, True)
            end_time = time.time()
            elapsed_time = end_time - start_time
            return ["Time taken:", elapsed_time, "seconds", "Best move: ", best_move, "Evaluation: ", best_value]
        else: # Now we make the engine think while it isn't it's turn ¡¡¡ACABAR ESTO!!!
            Dict = {}
            new_position = position.move(move)
            best_move = None
            Dict[move]=(new_engine_position(new_position, is_our_move, depth+1)[4], 
                        new_engine_position(new_position, is_our_move, depth+1)[6])
            return(Dict) # This is a dictionary of the form {move: best response, evaluation}
    def Tell_Move(move): # Spit move to UCI
        square = Squares_Dict[move.i]
        destination = Squares_Dict[move.j]
        return ''.join([square, destination, move.prom])


In [375]:
Engine.Tell_Move(Move(1,8,'Q',0))


'b1a2Q'

### Checkmates (The engine makes them all)

In [613]:
Engine.Search(fen_to_position('8/8/8/8/8/2R5/1R6/6k1 w - - 0 1'), True, 2)


['Time taken:',
 0.08354902267456055,
 'seconds',
 'Best move: ',
 Move(i=18, j=2, prom='', score=3),
 'Evaluation: ',
 1003]

In [624]:
print(Engine.Search(fen_to_position('6k1/4Rppp/8/8/8/8/5PPP/6K1 w - - 0 1'), True, 2))



['Time taken:', 0.0991671085357666, 'seconds', 'Best move: ', Move(i=52, j=60, prom='', score=3), 'Evaluation: ', 1003]


In [629]:
print(Engine.Search(fen_to_position('2r1r1k1/5ppp/8/8/Q7/8/5PPP/4R1K1 w - - 0 1'), True, 4))


['Time taken:', 0.7647218704223633, 'seconds', 'Best move: ', Move(i=4, j=60, prom='', score=5), 'Evaluation: ', 1001]


In [628]:
print(Engine.Search(fen_to_position('6k1/3qb1pp/4p3/ppp1P3/8/2PP1Q2/PP4PP/5RK1 w - - 0 1'), True, 4))


['Time taken:', 2.3391129970550537, 'seconds', 'Best move: ', Move(i=21, j=53, prom='', score=3), 'Evaluation: ', 1001]


In [630]:
print(Engine.Search(fen_to_position('R7/4kp2/5N2/4P3/8/8/8/6K1 w - - 0 1'), True, 4))


['Time taken:', 2.2835168838500977, 'seconds', 'Best move: ', Move(i=56, j=60, prom='', score=3), 'Evaluation: ', 1003]


In [739]:
print(Engine.Search(fen_to_position('5r1b/2R1R3/P4r2/2p2Nkp/2b3pN/6P1/4PP2/6K1 w - - 0 1'), True, 4))


['Time taken:', 50.26174783706665, 'seconds', 'Best move: ', Move(i=52, j=54, prom='', score=3), 'Evaluation: ', 1001]


### Some simple tactics

In [740]:
pretty_print(fen_to_position('r2qkbnr/1bp2ppp/1pnp4/pB2p3/3PP3/2N2N2/PPP2PPP/R1BQ1RK1 w kq - 2 7').board)
print(Engine.Search(fen_to_position('r2qkbnr/1bp2ppp/1pnp4/pB2p3/3PP3/2N2N2/PPP2PPP/R1BQ1RK1 w kq - 2 7'), True, 3), 'Best move should be: 27, 35')


♜..♛♚♝♞♜
.♝♟..♟♟♟
.♟♞♟....
♟♗..♟...
...♙♙...
..♘..♘..
♙♙♙..♙♙♙
♖.♗♕.♖♔.
['Time taken:', 25.392232179641724, 'seconds', 'Best move: ', Move(i=33, j=42, prom='', score=5), 'Evaluation: ', 600] Best move should be: 27, 35


In [741]:
pretty_print(fen_to_position('1kr2b1r/ppp2ppp/2n2n2/3p4/3P2qP/2N1P1P1/PPP2P2/R1BKQ2R w KQk - 0 1').board)
print(list(fen_to_position('1kr2b1r/ppp2ppp/2n2n2/3p4/3P2qP/2N1P1P1/PPP2P2/R1BKQ2R w KQk - 0 1').out_of_attack_moves()))
print(list(fen_to_position('1kr2b1r/ppp2ppp/2n2n2/3p4/3P2qP/2N1P1P1/PPP2P2/R1BKQ2R w KQk - 0 1').ordered_moves()))


.♚♜..♝.♜
♟♟♟..♟♟♟
..♞..♞..
...♟....
...♙..♛♙
..♘.♙.♙.
♙♙♙..♙..
♖.♗♔♕..♖
[Move(i=3, j=11, prom='', score=0), Move(i=4, j=12, prom='', score=0), Move(i=13, j=21, prom='', score=0), Move(i=18, j=12, prom='', score=0)]
[Move(i=3, j=11, prom='', score=0), Move(i=4, j=12, prom='', score=0), Move(i=13, j=21, prom='', score=0), Move(i=18, j=12, prom='', score=0)]


In [742]:
pretty_print(fen_to_position('1k6/ppp3q1/8/4r3/8/8/3B1PPP/R4QK1 w - - 0 1').board)
print(Engine.Search(fen_to_position('1k6/ppp3q1/8/4r3/8/8/3B1PPP/R4QK1 w - - 0 1'), True, 4), 'Best move should be: 11, 18')


.♚......
♟♟♟...♛.
........
....♜...
........
........
...♗.♙♙♙
♖....♕♔.
['Time taken:', 132.9972438812256, 'seconds', 'Best move: ', Move(i=11, j=18, prom='', score=0), 'Evaluation: ', 400] Best move should be: 11, 18


In [743]:
pretty_print(fen_to_position('4k3/6p1/5p1p/4n3/8/7P/5PP1/4R1K1 w - - 0 1').board)
print(Engine.Search(fen_to_position('4k3/6p1/5p1p/4n3/8/7P/5PP1/4R1K1 w - - 0 1'), True, 4), 'Best move should be: 13, 29')


....♚...
......♟.
.....♟.♟
....♞...
........
.......♙
.....♙♙.
....♖.♔.
['Time taken:', 2.7279059886932373, 'seconds', 'Best move: ', Move(i=13, j=29, prom='', score=0), 'Evaluation: ', 400] Best move should be: 13, 29


In [744]:
pretty_print(fen_to_position('r4rk1/pp1p1ppp/1qp2n2/8/4P3/1P1P2Q1/PBP2PPP/R4RK1 w - - 0 1').board)
print(Engine.Search(fen_to_position('r4rk1/pp1p1ppp/1qp2n2/8/4P3/1P1P2Q1/PBP2PPP/R4RK1 w - - 0 1'), True, 4), 'Best move should be: 9, 45')


♜....♜♚.
♟♟.♟.♟♟♟
.♛♟..♞..
........
....♙...
.♙.♙..♕.
♙♗♙..♙♙♙
♖....♖♔.
['Time taken:', 56.72650098800659, 'seconds', 'Best move: ', Move(i=9, j=45, prom='', score=2), 'Evaluation: ', 400] Best move should be: 9, 45


In [746]:
pretty_print(fen_to_position('2k2bnr/pp2pp1p/6p1/5n2/8/7B/1PP1P1PP/2B1K1NR w Kk - 0 1').board)
print(Engine.Search(fen_to_position('2k2bnr/pp2pp1p/6p1/5n2/8/7B/1PP1P1PP/2B1K1NR w Kk - 0 1'), True, 2), 'Best move should be: 12, 28')


..♚..♝♞♜
♟♟..♟♟.♟
......♟.
.....♞..
........
.......♗
.♙♙.♙.♙♙
..♗.♔.♘♖
['Time taken:', 0.4049570560455322, 'seconds', 'Best move: ', Move(i=12, j=28, prom='', score=0), 'Evaluation: ', 200] Best move should be: 12, 28


In [620]:
positionT = fen_to_position('2k2bnr/pp2pp1p/6p1/5n2/8/7B/1PP1P1PP/2B1K1NR w Kk - 0 1')
pretty_print(positionT.board)
print(positionT.ordered_moves())

..♚..♝♞♜
♟♟..♟♟.♟
......♟.
.....♞..
........
.......♗
.♙♙.♙.♙♙
..♗.♔.♘♖
[Move(i=23, j=37, prom='', score=5), Move(i=2, j=11, prom='', score=0), Move(i=2, j=20, prom='', score=0), Move(i=2, j=29, prom='', score=0), Move(i=2, j=38, prom='', score=0), Move(i=2, j=47, prom='', score=0), Move(i=4, j=3, prom='', score=0), Move(i=4, j=13, prom='', score=0), Move(i=4, j=11, prom='', score=0), Move(i=4, j=5, prom='', score=0), Move(i=6, j=21, prom='', score=0), Move(i=9, j=17, prom='', score=0), Move(i=9, j=25, prom='', score=0), Move(i=10, j=18, prom='', score=0), Move(i=10, j=26, prom='', score=0), Move(i=12, j=20, prom='', score=0), Move(i=12, j=28, prom='', score=0), Move(i=14, j=22, prom='', score=0), Move(i=14, j=30, prom='', score=0), Move(i=23, j=30, prom='', score=0)]


In [621]:
positionT.move(Move(12,28,'',0))
pretty_print(positionT.board)
print(positionT.ordered_moves())

♜♞.♚.♝..
♟♟...♟♟.
♝.......
...♟....
..♘.....
.♙......
♙.♙♙..♙♙
♖♘♗..♔..
[Move(i=1, j=18, prom='', score=0), Move(i=1, j=16, prom='', score=0), Move(i=2, j=9, prom='', score=0), Move(i=2, j=16, prom='', score=0), Move(i=5, j=4, prom='', score=0), Move(i=5, j=13, prom='', score=0), Move(i=5, j=12, prom='', score=0), Move(i=5, j=6, prom='', score=0), Move(i=8, j=16, prom='', score=0), Move(i=8, j=24, prom='', score=0), Move(i=10, j=18, prom='', score=0), Move(i=11, j=19, prom='', score=0), Move(i=11, j=27, prom='', score=0), Move(i=14, j=22, prom='', score=0), Move(i=14, j=30, prom='', score=0), Move(i=15, j=23, prom='', score=0), Move(i=15, j=31, prom='', score=0), Move(i=17, j=25, prom='', score=0)]


In [622]:
positionT.move(Move(11,20,'',0))
pretty_print(positionT.board)
print(positionT.ordered_moves())

..♚..♝♞♜
♟♟...♟.♟
...♟..♟.
.....♞..
....♙...
.......♗
.♙♙...♙♙
..♗.♔.♘♖
[Move(i=23, j=37, prom='', score=5), Move(i=28, j=37, prom='', score=2), Move(i=2, j=11, prom='', score=0), Move(i=2, j=20, prom='', score=0), Move(i=2, j=29, prom='', score=0), Move(i=2, j=38, prom='', score=0), Move(i=2, j=47, prom='', score=0), Move(i=4, j=3, prom='', score=0), Move(i=4, j=13, prom='', score=0), Move(i=4, j=12, prom='', score=0), Move(i=4, j=11, prom='', score=0), Move(i=4, j=5, prom='', score=0), Move(i=6, j=21, prom='', score=0), Move(i=6, j=12, prom='', score=0), Move(i=9, j=17, prom='', score=0), Move(i=9, j=25, prom='', score=0), Move(i=10, j=18, prom='', score=0), Move(i=10, j=26, prom='', score=0), Move(i=14, j=22, prom='', score=0), Move(i=14, j=30, prom='', score=0), Move(i=23, j=30, prom='', score=0), Move(i=28, j=36, prom='', score=0)]


In [623]:
positionT.move(Move(28,37,'',0))
pretty_print(positionT.board)
print(positionT.ordered_moves())

♜♞.♚.♝..
♟♟...♟♟.
♝.......
........
..♟.....
.♙..♙...
♙.♙...♙♙
♖♘♗..♔..
[Move(i=17, j=26, prom='', score=1), Move(i=1, j=18, prom='', score=0), Move(i=1, j=11, prom='', score=0), Move(i=1, j=16, prom='', score=0), Move(i=2, j=11, prom='', score=0), Move(i=2, j=9, prom='', score=0), Move(i=2, j=16, prom='', score=0), Move(i=5, j=4, prom='', score=0), Move(i=5, j=13, prom='', score=0), Move(i=5, j=12, prom='', score=0), Move(i=5, j=6, prom='', score=0), Move(i=8, j=16, prom='', score=0), Move(i=8, j=24, prom='', score=0), Move(i=10, j=18, prom='', score=0), Move(i=14, j=22, prom='', score=0), Move(i=14, j=30, prom='', score=0), Move(i=15, j=23, prom='', score=0), Move(i=15, j=31, prom='', score=0), Move(i=17, j=25, prom='', score=0), Move(i=20, j=28, prom='', score=0)]


### A game

In [811]:
p1 = fen_to_position('rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1')
pretty_print(p1.board)
pretty_print(p1.move(Move(14, 22, '',0)).board)

♜♞♝♛♚♝♞♜
♟♟♟♟♟♟♟♟
........
........
........
........
♙♙♙♙♙♙♙♙
♖♘♗♕♔♗♘♖
♜♞♝♚♛♝♞♜
♟.♟♟♟♟♟♟
.♟......
........
........
........
♙♙♙♙♙♙♙♙
♖♘♗♔♕♗♘♖


In [812]:
pretty_print(p1.board)
print(Engine.Search(p1,True,2))

♜♞♝♚♛♝♞♜
♟.♟♟♟♟♟♟
.♟......
........
........
........
♙♙♙♙♙♙♙♙
♖♘♗♔♕♗♘♖
['Time taken:', 0.2939901351928711, 'seconds', 'Best move: ', Move(i=1, j=18, prom='', score=0), 'Evaluation: ', 0]


In [813]:
p1.move(Move(6,21,'',0))
pretty_print(p1.board)

♜.♝♛♚♝♞♜
♟♟♟♟♟♟♟♟
..♞.....
........
........
......♙.
♙♙♙♙♙♙.♙
♖♘♗♕♔♗♘♖


In [814]:
p1.move(Move(9,17,'',0))
print(Engine.Search(p1,True,2))

['Time taken:', 0.3251171112060547, 'seconds', 'Best move: ', Move(i=1, j=18, prom='', score=0), 'Evaluation: ', 0]


In [815]:
p1.move(Move(1,16,'',0))
pretty_print(p1.board)

♜.♝♛♚♝.♜
♟♟♟♟♟♟♟♟
..♞....♞
........
........
.♙....♙.
♙.♙♙♙♙.♙
♖♘♗♕♔♗♘♖


In [816]:
p1.move(Move(8,16,'',0))
pretty_print(p1.board)

♜♞♝♚♛♝♞♜
♟.♟♟♟♟..
.♟....♟♟
........
........
♘....♘..
♙♙♙♙♙♙♙♙
♖.♗♔♕♗.♖


In [817]:
print(Engine.Search(p1,True,2))

['Time taken:', 0.30702996253967285, 'seconds', 'Best move: ', Move(i=0, j=1, prom='', score=0), 'Evaluation: ', 0]


In [818]:
p1.move(Move(7,6,'',0))
pretty_print(p1.board)

.♜♝♛♚♝.♜
♟♟♟♟♟♟♟♟
..♞....♞
........
........
♙♙....♙.
..♙♙♙♙.♙
♖♘♗♕♔♗♘♖


In [819]:
p1.move(Move(17,25,'',0))
pretty_print(p1.board)

♜♞♝♚♛♝♞♜
♟.♟♟♟♟..
.♟.....♟
......♟.
........
♘....♘..
♙♙♙♙♙♙♙♙
♖.♗♔♕♗♖.


In [820]:
print(Engine.Search(p1,True,2))

['Time taken:', 0.5141639709472656, 'seconds', 'Best move: ', Move(i=0, j=1, prom='', score=0), 'Evaluation: ', 0]


In [821]:
p1.move(Move(0,1,'',0))
pretty_print(p1.board)

.♜♝♛♚♝♜.
♟♟♟♟♟♟♟♟
..♞....♞
........
.♙......
♙.....♙.
..♙♙♙♙.♙
♖♘♗♕♔♗♘♖


In [822]:
p1.move(Move(1,18,'',0))
pretty_print(p1.board)

♜♞♝♚♛♝.♜
♟.♟♟♟♟..
.♟...♞.♟
......♟.
........
♘....♘..
♙♙♙♙♙♙♙♙
.♖♗♔♕♗♖.


In [823]:
print(Engine.Search(p1,True,2))

['Time taken:', 0.5940768718719482, 'seconds', 'Best move: ', Move(i=1, j=0, prom='', score=0), 'Evaluation: ', 0]


In [824]:
p1.move(Move(6,7,'',0))
pretty_print(p1.board)

♜.♝♛♚♝♜.
♟♟♟♟♟♟♟♟
..♞....♞
........
.♙......
♙.♘...♙.
..♙♙♙♙.♙
♖.♗♕♔♗♘♖


In [825]:
p1.move(Move(18,28,'',0))
pretty_print(p1.board)

♜♞♝♚♛♝.♜
♟.♟♟♟♟..
.♟.....♟
...♞..♟.
........
♘....♘..
♙♙♙♙♙♙♙♙
.♖♗♔♕♗.♖


In [826]:
print(Engine.Search(p1,True,2))

['Time taken:', 1.492875099182129, 'seconds', 'Best move: ', Move(i=1, j=0, prom='', score=0), 'Evaluation: ', 0]


In [827]:
p1.move(Move(14,30,'',0))
pretty_print(p1.board)

♜.♝♛♚♝♜.
♟.♟♟♟♟♟♟
..♞....♞
.♟......
.♙..♘...
♙.....♙.
..♙♙♙♙.♙
♖.♗♕♔♗♘♖


In [828]:
p1.move(Move(15,31,'',0))
pretty_print(p1.board)

♜♞♝♚♛♝.♜
..♟♟♟♟..
.♟.....♟
♟..♞..♟.
......♙.
♘....♘..
♙♙♙♙♙♙.♙
.♖♗♔♕♗.♖


In [829]:
print(Engine.Search(p1,True,2))

['Time taken:', 1.5916788578033447, 'seconds', 'Best move: ', Move(i=1, j=0, prom='', score=0), 'Evaluation: ', 0]


In [830]:
p1.move(Move(5,23,'',0))
pretty_print(p1.board)

♜..♛♚♝♜.
♟.♟♟♟♟♟♟
♝.♞....♞
.♟......
.♙..♘..♙
♙.....♙.
..♙♙♙♙..
♖.♗♕♔♗♘♖


In [831]:
p1.move(Move(28,34,'',0))
pretty_print(p1.board)

♜♞♝♚♛♝.♜
..♟♟♟♟..
.♟.....♟
♟.....♟.
.....♞♙.
♘....♘.♗
♙♙♙♙♙♙.♙
.♖♗♔♕..♖


In [832]:
print(Engine.Search(p1,True,3))

['Time taken:', 8.90531611442566, 'seconds', 'Best move: ', Move(i=4, j=5, prom='', score=0), 'Evaluation: ', 0]


In [833]:
p1.move(Move(4,5,'',0))
pretty_print(p1.board)

♜.♛.♚♝♜.
♟.♟♟♟♟♟♟
♝.♞....♞
.♟♘.....
.♙.....♙
♙.....♙.
..♙♙♙♙..
♖.♗♕♔♗♘♖


In [834]:
p1.move(Move(34,40,'',0))
pretty_print(p1.board)

♜♞♝♚♛♝.♜
..♟♟♟♟..
.♟.....♟
♟.....♟.
......♙.
♘....♘.♞
♙♙♙♙♙♙.♙
.♖♗♔.♕.♖


In [835]:
print(Engine.Search(p1,True,2))

['Time taken:', 0.36885809898376465, 'seconds', 'Best move: ', Move(i=5, j=23, prom='', score=2), 'Evaluation: ', 0]


In [836]:
p1.move(Move(5,23,'',0))
pretty_print(p1.board)

♜...♚♝♜.
♟.♟♟♟♟♟♟
♛.♞....♞
.♟......
.♙.....♙
♙.....♙.
..♙♙♙♙..
♖.♗♕♔♗♘♖


In [794]:
p1.move(Move(5,23,'',0))
pretty_print(p1.board)

♜♞.♚♛♝.♜
..♟♟♟♟..
♝♟.....♟
♟.....♟.
......♙.
♘......♘
♙♙♙♙♙♙.♙
.♖♗♔♕..♖


In [795]:
print(Engine.Search(p1,True,2))

['Time taken:', 0.7454116344451904, 'seconds', 'Best move: ', Move(i=1, j=0, prom='', score=0), 'Evaluation: ', 0]


In [796]:
p1.move(Move(12,20,'',0))
pretty_print(p1.board)

♜..♛♚♝♜.
♟.♟.♟♟♟♟
♞..♟...♞
.♟......
.♙.....♙
♙.....♙♗
..♙♙♙♙..
♖.♗♕♔.♘♖


In [797]:
p1.move(Move(6,21,'',0))
pretty_print(p1.board)

♜..♚♛♝.♜
..♟♟♟♟..
♝♟♞....♟
♟.....♟.
......♙.
♘...♙..♘
♙♙♙♙.♙.♙
.♖♗♔♕..♖


In [798]:
print(Engine.Search(p1,True,3))

['Time taken:', 13.601054906845093, 'seconds', 'Best move: ', Move(i=9, j=25, prom='', score=0), 'Evaluation: ', 100]


In [799]:
p1.move(Move(4,6,'',0))
pretty_print(p1.board)

♜♛..♚♝♜.
♟.♟.♟♟♟♟
♞..♟...♞
.♟......
.♙.....♙
♙....♘♙♗
..♙♙♙♙..
♖.♗♕♔..♖


In [800]:
p1.move(Move(2,9,'',0))
pretty_print(p1.board)

♜..♚♛..♜
..♟♟♟♟♝.
♝♟♞....♟
♟.....♟.
......♙.
♘...♙..♘
♙♙♙♙.♙.♙
.♖♗♔..♕♖


In [801]:
print(Engine.Search(p1,True,4))

['Time taken:', 106.63696098327637, 'seconds', 'Best move: ', Move(i=11, j=27, prom='', score=0), 'Evaluation: ', 0]


In [802]:
p1.move(Move(1,0,'',0))
pretty_print(p1.board)

♜♛..♚♝.♜
♟.♟.♟♟♟♟
♞..♟...♞
.♟......
.♙.....♙
♙....♘♙♗
.♗♙♙♙♙..
♖..♕♔..♖


In [803]:
p1.move(Move(12,28,'',0))
pretty_print(p1.board)

♜..♚♛..♜
..♟.♟♟♝.
♝♟♞....♟
♟..♟..♟.
......♙.
♘...♙..♘
♙♙♙♙.♙.♙
♖.♗♔..♕♖


In [804]:
print(Engine.Search(p1,True,3))

['Time taken:', 10.423593997955322, 'seconds', 'Best move: ', Move(i=0, j=1, prom='', score=0), 'Evaluation: ', 0]


In [805]:
p1.move(Move(0,1,'',0))
pretty_print(p1.board)

♜♛..♚♝♜.
♟.♟.♟♟♟♟
♞..♟...♞
.♟......
.♙..♙..♙
♙....♘♙♗
.♗♙♙.♙..
♖..♕♔..♖


In [806]:
p1.move(Move(11,27,'',0))
pretty_print(p1.board)

♜..♚♛..♜
..♟..♟♝.
♝♟♞....♟
♟..♟♟.♟.
......♙.
♘...♙..♘
♙♙♙♙.♙.♙
.♖♗♔..♕♖


In [807]:
print(Engine.Search(p1,True,3))

['Time taken:', 10.924126863479614, 'seconds', 'Best move: ', Move(i=1, j=0, prom='', score=0), 'Evaluation: ', 0]


In [808]:
p1.move(Move(1,0,'',0))
pretty_print(p1.board)

♜♛..♚♝.♜
♟.♟.♟♟♟♟
♞..♟...♞
.♟......
.♙.♙♙..♙
♙....♘♙♗
.♗♙..♙..
♖..♕♔..♖


In [809]:
p1.move(Move(9,2,'',0))
pretty_print(p1.board)

♜..♚♛♝.♜
..♟..♟..
♝♟♞....♟
♟..♟♟.♟.
......♙.
♘...♙..♘
♙♙♙♙.♙.♙
♖.♗♔..♕♖


In [810]:
print(Engine.Search(p1,True,4))

['Time taken:', 228.79460191726685, 'seconds', 'Best move: ', Move(i=0, j=1, prom='', score=0), 'Evaluation: ', -100]


## UCI

Now I'm going to create a function that using the position encoder, also gets the history matrices and repetition matrices.

In [ ]:
def max_string_count(str_list):
    counts = {}
    for s in str_list:
        if s in counts:
            counts[s] += 1
        else:
            counts[s] = 1
    return max(counts.values())
# I HAVE CHECKED THAT THIS ONE IS RUNNING CORRECTLY

In [ ]:
def game_encoder(pgn):
    hist_1 = [] #most recent position
    hist_2 = []
    hist_3 = []
    hist_4 = []
    hist_5 = []
    hist_6 = [] #6th most recent position
    for i in range(len(encode_position('rnbqkbnr/pppppppp/8/8/8/1P6/P1PPPPPP/RNBQKBNR b KQkq - 0 1'))):
        hist_1.append(np.zeros((8,8)))
        hist_2.append(np.zeros((8,8)))
        hist_3.append(np.zeros((8,8)))
        hist_4.append(np.zeros((8,8)))
        hist_5.append(np.zeros((8,8)))
        hist_6.append(np.zeros((8,8)))
    histories = [hist_1, hist_2, hist_3, hist_4, hist_5, hist_6]
    Fens = pgn_to_fenslist(pgn)
    for fen in Fens:
        histories[5] = histories[4]
        histories[4] = histories[3]
        histories[3] = histories[2]
        histories[2] = histories[1]
        histories[1] = histories[0]
        histories[0] = encode_position(fen)
    matrices_position = encode_position(Fens[-1])
    r = max_string_count(Fens)-1
    if r == 0:
        R_1 = np.zeros((8,8))
        R_2 = np.zeros((8,8))
    if r == 1:
        R_1 = np.ones((8,8))
        R_2 = np.zeros((8,8))
    else:
        R_1 = np.zeros((8,8))
        R_2 = np.ones((8,8))
    klk = [matrices_position, R_1, R_2, hist_1, hist_2, hist_3, hist_4, hist_5, hist_6]
    flat_list = [item for sublist in klk for item in sublist]
    return flat_list

In [ ]:
def game_encoder_for_training(pgn):
    hist_1 = [] #most recent position
    hist_2 = []
    hist_3 = []
    hist_4 = []
    hist_5 = []
    hist_6 = [] #6th most recent position
    for i in range(len(encode_position('rnbqkbnr/pppppppp/8/8/8/1P6/P1PPPPPP/RNBQKBNR b KQkq - 0 1'))):
        hist_1.append(np.zeros((8,8)))
        hist_2.append(np.zeros((8,8)))
        hist_3.append(np.zeros((8,8)))
        hist_4.append(np.zeros((8,8)))
        hist_5.append(np.zeros((8,8)))
        hist_6.append(np.zeros((8,8)))
    histories = [hist_1, hist_2, hist_3, hist_4, hist_5, hist_6]
    Dict = pgneval_to_dict(pgn)
    for fen in Dict.keys():
        histories[5] = histories[4]
        histories[4] = histories[3]
        histories[3] = histories[2]
        histories[2] = histories[1]
        histories[1] = histories[0]
        histories[0] = encode_position(fen)
    matrices_position = encode_position(list(Dict.keys())[-1])
    r = max_string_count(list(Dict.keys()))-1
    if r == 0:
        R_1 = np.zeros((8,8))
        R_2 = np.zeros((8,8))
    if r == 1:
        R_1 = np.ones((8,8))
        R_2 = np.zeros((8,8))
    else:
        R_1 = np.zeros((8,8))
        R_2 = np.ones((8,8))
    
    klk = [encode_position(list(Dict.keys())[-1]), R_1, R_2, hist_1, hist_2, hist_3, hist_4, hist_5, hist_6]
    flat_list = [item for sublist in klk for item in sublist]
    return flat_list

In [ ]:
len(game_encoder('Carlsen - Martirosyan eval.pgn'))

In [ ]:
len(game_encoder_for_training('Carlsen - Martirosyan eval.pgn'))